### Napari Viewer 
Napari can be used as a GUI for registration with elastix.
This can only be run locally, since napari opens an external window.

In [1]:
import enum
import napari
from napari.layers import Image
from magicgui import magicgui
from qtpy.QtGui import QFont
from itk import itkElastixRegistrationMethodPython
import itk
from itkwidgets import compare, checkerboard
import numpy as np

In [2]:
# Load images with itk floats (itk.F). Necessary for elastix
fixed_image = itk.imread('data/CT_2D_head_fixed.mha', itk.F)
moving_image = itk.imread('data/CT_2D_head_moving.mha', itk.F)

# Cast images to numpy arrays for Napari Viewer
fixed_image_np = np.asarray(fixed_image).astype(np.float32)
moving_image_np = np.asarray(moving_image).astype(np.float32)

In [9]:
# Create parameterObject function
def parameterObject(transform, resolution, optimizer, imageSampler):
    
    parameter_object = itk.ParameterObject.New()
    default_rigid_parameter_map = parameter_object.GetDefaultParameterMap(transform, resolution)
    default_rigid_parameter_map['Optimizer'] = [optimizer]
    default_rigid_parameter_map['ImageSampler'] = [imageSampler]
    parameter_object.AddParameterMap(default_rigid_parameter_map)
    
    return parameter_object

In [10]:
# Create Napari Gui plugin
with napari.gui_qt():
    viewer = napari.Viewer()
    viewer.add_image(fixed_image_np, name=f"Fixed")
    viewer.add_image(moving_image_np, name=f"Moving")

    @magicgui(call_button="register", 
              transform = {"choices": ["rigid", "affine", "bspline"]},
              resolutions = {"resolutions": int, "minimum":1, "maximum":10},
              optimizer = {"choices":["AdaptiveStochasticGradientDescent","ConjugateGradient","FiniteDifferenceGradientDescent","Powell","PreconditionedStochasticGradientDescent","QuasiNewtonLBFGS","RegularStepGradientDescent", "StandardGradientDescent"]},
              imageSampler = {"choices":["Full", "Grid", "RandomCoordinate", "RandomSparseMask", "Random"]})
    def registration(transform= "rigid", resolution = 2, optimizer = "AdaptiveStochasticGradientDescent", imageSampler="Random") -> Image:
        
        parameter_object = parameterObject(transform, resolution, optimizer, imageSampler)
        result_image, result_transform_parameters = itk.elastix_registration_method(
            fixed_image_np, moving_image_np,
            parameter_object=parameter_object,
            log_to_console=True)
        result_image_np = np.asarray(result_image).astype(np.float32)
        
        viewer.add_image(result_image_np, name=f"Result_"+transform+"_"+imageSampler)
        return result_image_np
        
    gui = registration.Gui()
    gui.setFont(QFont('Arial',15))
    viewer.window.add_dock_widget(gui)
    viewer.layers.events.changed.connect(lambda x: gui.refresh_choices())